<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NPA: Neural News Recommendation with Personalized Attention
NPA \[1\] is a news recommendation model with personalized attention. The core of NPA is a news representation model and a user representation model. In the news representation model we use a CNN network to learn hidden representations of news articles based on their titles. In the user representation model we learn the representations of users based on the representations of their clicked news articles. In addition, a word-level and a news-level personalized attention are used to capture different informativeness for different users.

## Properties of NPA:
- NPA is a content-based news recommendation method.
- It uses a CNN network to learn news representation. And it learns user representations from their clicked news articles.
- A word-level personalized attention is used to help NPA attend to important words for different users.
- A news-level personalized attention is used to help NPA attend to important historical clicked news for different users.

## Data format:

### train data
One simple example: <br>

`1 0 0 0 0 Impression:0 User:2903 CandidateNews0:27006,11901,21668,9856,16156,21390,1741,2003,16983,8164 CandidateNews1:8377,10423,9960,5485,20494,7553,1251,17232,4745,9178 CandidateNews2:1607,26414,25830,16156,15337,16461,4004,6230,17841,10704 CandidateNews3:17323,20324,27855,16156,2934,14673,551,0,0,0 CandidateNews4:7172,3596,25442,21596,26195,4745,17988,16461,1741,76 ClickedNews0:11362,8205,22501,9349,12911,20324,1238,11362,26422,19185 ...`
<br>

In general, each line in data file represents one positive instance and n negative instances in a same impression. The format is like: <br>

`[label0] ... [labeln] [Impression:i] [User:u] [CandidateNews0:w1,w2,w3,...] ... [CandidateNewsn:w1,w2,w3,...] [ClickedNews0:w1,w2,w3,...] ...`

<br>

It contains several parts seperated by space, i.e. label part, Impression part `<impresison id>`, User part `<user id>`, CandidateNews part, ClickedHistory part. CandidateNews part describes the target news article we are going to score in this instance, it is represented by (aligned) title words. To take a quick example, a news title may be : `Trump to deliver State of the Union address next week` , then the title words value may be `CandidateNewsi:34,45,334,23,12,987,3456,111,456,432`. <br>
ClickedNewsk describe the k-th news article the user ever clicked and the format is the same as candidate news. Words are aligned in news title. We use a fixed length to describe an article, if the title is less than the fixed length, just pad it with zeros.

### test data
One simple example: <br>
`1 Impression:0 User:6446 CandidateNews0:18707,23848,13490,10948,21385,11606,1251,16591,827,28081 ClickedNews0:27838,7376,16567,28518,119,21248,7598,9349,20324,9349 ClickedNews1:7969,9783,1741,2549,27104,14669,14777,21343,7667,20324 ...`
<br>

In general, each line in data file represents one instance. The format is like: <br>

`[label] [Impression:i] [User:u] [CandidateNews0:w1,w2,w3,...] [ClickedNews0:w1,w2,w3,...] ...`
<br>

## Global settings and imports

In [2]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources 
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.npa import NPAModel
from reco_utils.recommender.newsrec.IO.news_iterator import NewsIterator
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]
Tensorflow version: 1.12.0


## Download and load data

In [3]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'nrms.yaml')
train_file = os.path.join(data_path, r'train.txt')
valid_file = os.path.join(data_path, r'test.txt')
wordEmb_file = os.path.join(data_path, r'embedding.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/newsrec/', data_path, 'nrms.zip')

100%|██████████| 21.2k/21.2k [00:02<00:00, 9.06kKB/s]


## Create hyper-parameters

In [4]:
epoch=5

In [5]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, epochs=epoch)
print(hparams)

[('attention_hidden_dim', 200), ('batch_size', 64), ('body_size', None), ('cnn_activation', None), ('data_format', 'news'), ('dense_activation', None), ('doc_size', 10), ('dropout', 0.2), ('epochs', 5), ('filter_num', 200), ('gru_unit', 400), ('head_dim', 50), ('head_num', 3), ('his_size', 50), ('iterator_type', None), ('learning_rate', 0.0001), ('loss', 'cross_entropy_loss'), ('metrics', ['group_auc', 'mean_mrr', 'ndcg@5;10']), ('npratio', 4), ('optimizer', 'adam'), ('show_step', 100000), ('subvert_emb_dim', 100), ('subvert_num', None), ('title_size', None), ('type', 'ini'), ('user_emb_dim', 50), ('user_num', 10338), ('vert_emb_dim', 100), ('vert_num', None), ('window_size', 3), ('wordEmb_file', '/tmp/tmp7q6e8t4e/embedding.npy'), ('word_emb_dim', 100), ('word_size', 28929)]


In [6]:
iterator = NewsIterator

## Train the NPA model

In [11]:
model = NPAModel(hparams, iterator)

In [12]:
print(model.run_eval(valid_file))

{'group_auc': 0.5134, 'mean_mrr': 0.1708, 'ndcg@5': 0.163, 'ndcg@10': 0.2183}


In [13]:
model.fit(train_file, valid_file)

/data/anaconda/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


at epoch 1
train info: logloss loss:1.6590138756498998
eval info: group_auc:0.5844, mean_mrr:0.1906, ndcg@10:0.2657, ndcg@5:0.1966
at epoch 1 , train time: 13.0 eval time: 8.1
at epoch 2
train info: logloss loss:1.5676321803307047
eval info: group_auc:0.5889, mean_mrr:0.1943, ndcg@10:0.2689, ndcg@5:0.1945
at epoch 2 , train time: 10.4 eval time: 8.3
at epoch 3
train info: logloss loss:1.5324120550739522
eval info: group_auc:0.5794, mean_mrr:0.1839, ndcg@10:0.2604, ndcg@5:0.1846
at epoch 3 , train time: 10.4 eval time: 8.2
at epoch 4
train info: logloss loss:1.3894470263500602
eval info: group_auc:0.5802, mean_mrr:0.195, ndcg@10:0.2704, ndcg@5:0.1936
at epoch 4 , train time: 10.4 eval time: 8.3
at epoch 5
train info: logloss loss:1.2671602429175863
eval info: group_auc:0.5869, mean_mrr:0.1992, ndcg@10:0.2723, ndcg@5:0.2032
at epoch 5 , train time: 10.6 eval time: 8.1


In [14]:
print(model.run_eval(valid_file))

{'group_auc': 0.5869, 'mean_mrr': 0.1992, 'ndcg@5': 0.2032, 'ndcg@10': 0.2723}


## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: NPA: Neural News Recommendation with Personalized Attention, KDD 2019, ADS track.<br>